In [1]:
# Alfredo Rojas
# 10/17/2020
# description: NDVI extraction, inspired by Qiusheng Wu's GEE tutorial #12
# filename: gee_py_ndvi.ipynb

import ee
import geemap
import os

TODO: Upload ESRI shp file. Create buffer of 10km and use that as Zonal Stats. . . 

In [52]:
Map = geemap.Map()
Map.setCenter(-2.142, 12.36)
Map

Map(center=[12.36, -2.142], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

In [43]:
### for 1993 DHS ###
#start_date = '1988-01-01'
#end_date = '1994-12-31'

### for 1998-99 DHS ###
#start_date = '1993-01-01'
#end_date = '2000-12-31'

### for 2003 DHS ###
#start_date = '1998-01-01'
#end_date = '2004-12-31'

### for 2010 DHS ###
start_date = '2005-01-01'
end_date = '2011-12-31'

# Figure out how to filter by ROI -- was getting Layer Error when filterBounds(states) was run. . .
# roi = ee.Geometry.Point([-96.86, 40.12])

# Add Earth Engine dataset, select NDVI + filter
dataset = ee.ImageCollection('NASA/GIMMS/3GV0') \
        .filterDate(start_date, end_date) 

ndvi = dataset.select('ndvi')

#Set visualization parameters.
ndvi_vis = {
  'min': -1,
  'max': 1,
  'palette': ['000000', 'f5f5f5', '119701']}

# Add Earth Engine AVHRR ndvi to map
Map.addLayer(ndvi, ndvi_vis, 'NDVI')

# bring in shape files
cluster_shp = "../Articles/HAZ_paper/DHS_Data_CI_BF/BF_Data/geographic/\
BF_2010_DHS_04032020_1546_134297/BFGE61FL/BFGE61FL.shp"

clusters = geemap.shp_to_ee(cluster_shp)
Map.addLayer(clusters, {}, 'DHSID')

# function to add 10km buffers around DHS clusters
def addBuffer(feature):
    return feature.buffer(10000)

# apply buffer and map
buffers = clusters.map(addBuffer)
Map.addLayer(buffers, {}, 'buffer')

There are some erroneous data points. . . with Lat/Lon = (0,0). Look at DHS manual to see what this means? 

In [44]:
print(clusters.size().getInfo())

573


In [45]:
# In order to do Zonal Stats on NDVI time series, use collection.toBands():
ndvi_bands = ndvi.toBands()

Note that GEE uses scale during calculations. In Zonal Stats below, you should specify Nominal Scale (resolution in meters) to avoid ambiguity. Methods to get nominal scale are shown below in print() statement. See [here](https://developers.google.com/earth-engine/tutorials/tutorial_api_03)

In [48]:
# get Nonimal Scale
print(ndvi_bands.projection().nominalScale().getInfo())

#print(ndvi.size().getInfo())
#print(ndvi_bands.bandNames().getInfo())
#print(ndvi.bandTypes().getInfo())
#type(clusers)
#print(buffers.size().getInfo())
#print(clusters.getInfo())

9276.624232772796


In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_ndvi_stats = os.path.join(out_dir, 'ndvi_05to11_mean.csv')  
geemap.zonal_statistics(ndvi_bands, buffers, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'ndvi_05to11_max.csv')  
geemap.zonal_statistics(ndvi_bands, buffers, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'ndvi_05to11_min.csv')  
geemap.zonal_statistics(ndvi_bands, buffers, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
#geemap.zonal_statistics(ndvi_bands, buffers, out_ndvi_stats, statistics_type='MEAN', scale=1000)